In [43]:
# testing 
import warnings
warnings.filterwarnings("ignore")
import os, time
from cvtk.core import Context, ObjectDetectionDataset, TFFasterRCNN
from cvtk.utils import detection_utils

# Disable printing of logging messages
from azuremltkbase.logging import ToolkitLogger
ToolkitLogger.getInstance().setEnabled(False)

# Initialize the context object
out_root_path = "../../../cvtk_output_test2"
Context.create(outputs_path=out_root_path, persistent_path=out_root_path, temp_path=out_root_path)

from matplotlib import pyplot as plt
# Display the images
%matplotlib inline

In [44]:
# initialize variables
image_folder = r"../sample_data/"
annotations_dir = "Annotation_100"
image_subdirectory = 'Data-100'

# training parameters
score_threshold = 0.0       # Threshold on the detection score, use to discard lower-confidence detections.
max_total_detections = 300  # Maximum number of detections. A high value will slow down training but might increase accuracy.

# to get good results, use a larger value for num_steps, e.g., 5000.
num_steps = 1
learning_rate = 0.001 # learning rate

# matfile name fir original labels
matfile_name = 'nyu_training_data_new.mat'

test_dirname = r'C:\Users\admin-dsvm\Downloads\imagesAndGt\test_images'
Out_csvFilename = r'Image_analysis_Inference_100.csv'

# save only while training
saved_model_path ="../../../cvtk_output/frozen_model/faster_rcnn_100.model" # Please save your model to outside of your AML workbench project folder because of the size limit of AML project


# set deployment name
deployment_name = "wsdeployment"

In [6]:
# training

data_train = ObjectDetectionDataset.create_from_dir(dataset_name='training_dataset', data_dir=image_folder,
                                                    annotations_dir=annotations_dir, image_subdirectory=image_subdirectory)

# Show some statistics of the training image, and also give one example of the ground truth rectangle annotations
data_train.print_info()
# _ = data_train.images[2].visualize_bounding_boxes(image_size = (10,10))

F1 2018-07-25 13:09:09,886 INFO azureml.vision:dataset creating dataset for scenario=detection 
Dataset name: training_dataset
Total classes: 10, total images: 100
Label-wise object counts:
	Label basket: 15 objects
	Label dishwasher: 12 objects
	Label headboard: 7 objects
	Label paper towel: 10 objects
	Label person: 21 objects
	Label plate: 22 objects
	Label printer: 21 objects
	Label remote control: 8 objects
	Label stuffed animal: 4 objects
	Label toilet: 6 objects
Bounding box width and height distribution:
	Bounding box widths  0/5/25/50/75/95/100-percentile: 14/30/57/90/176/299/619 pixels
	Bounding box heights 0/5/25/50/75/95/100-percentile: 4/12/41/86/160/281/421 pixels


In [7]:

my_detector = TFFasterRCNN(labels=data_train.labels, 
                           score_threshold=score_threshold, 
                           max_total_detections=max_total_detections)

In [8]:
print(my_detector.class_map)

None


In [9]:
# single step for testing  to get classes
print("tensorboard --logdir={}".format(my_detector.train_dir))

start_train = time.time()
my_detector.train(dataset=data_train, num_steps=num_steps, 
                  initial_learning_rate=learning_rate)
end_train = time.time()
print(end_train-start_train)

tensorboard --logdir=C:\Users\admin-dsvm\Documents\cvp_project\cvtk_output_test1\temp_faster_rcnn_resnet50\models\train
F1 2018-07-25 13:09:58,346 INFO azureml.vision:Fit starting in experiment 894827985 
F1 2018-07-25 13:09:58,349 INFO azureml.vision:model starting trainging for scenario=detection 

The checkpoint files are available from Tensorflow.org under the Google Terms of Service (https://policies.google.com/terms).

Download complete!
TFRecords creation started.
F1 2018-07-25 13:10:24,374 INFO On image 0 of 100
TFRecords creation completed.
Training started.
Training progressing: step 0 ...
F1 2018-07-25 13:11:37,516 INFO Graph Rewriter optimizations enabled
Converted 275 variables to const ops.
F1 2018-07-25 13:11:45,313 INFO 3159 ops in the final graph.
F1 2018-07-25 13:11:58,994 INFO azureml.vision:Fit finished in experiment 894827985 
Training completed.
120.65305805206299


In [10]:
print("tensorboard --logdir={} --port=8008".format(my_detector.eval_dir))

tensorboard --logdir=C:\Users\admin-dsvm\Documents\cvp_project\cvtk_output_test1\temp_faster_rcnn_resnet50\models\eval --port=8008


In [11]:
####################### inference #####################

In [12]:
# testing
import scipy.io as sio
import numpy as np
import os
import csv

In [13]:
def create_names_list(names):
    nms = []
    for name in names:
        nms.append(name[0][0])
    return nms

In [14]:
# Reading Original label csv file and adding the data to actual 
actual = []
with open(r'C:\Users\admin-dsvm\Downloads\imagesAndGt\GroundTruth.csv', 'r') as gt_csv:
    csv_reader1 = csv.reader(gt_csv)
    next(gt_csv)
    for lines in csv_reader1:
        actual.append(lines)

In [45]:

my_detector_loaded = TFFasterRCNN.load(saved_model_path)

In [46]:

listdir = os.listdir(test_dirname)
# print(listdir)

In [47]:
print(my_detector.class_map)

{'toilet': 5, 'remote control': 8, 'person': 3, 'dishwasher': 1, 'printer': 2, 'headboard': 7, 'plate': 10, 'stuffed animal': 9, 'basket': 6, 'paper towel': 4}


In [48]:
look_up = dict((v,k) for k,v in my_detector.class_map.items())
with open(Out_csvFilename, 'w') as csvfile:
        fieldnames = ['FileName', 'Predicted_Lable', 'Original_Label', 'Predicted_Correct_Label',
                              'Accuracy']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for idx, name in enumerate(listdir):
            
            print(name)
            
            ind = int(name.split('.jpg')[0].split('_')[1])
            act = actual[ind][1]
    
            
            # get the image_path. And detect the image
            image_path = test_dirname + '/' + name
            
            detections_dict = my_detector_loaded.score(image_path)
            
            
            n_obj = 0
            pred_labels = []
#             print(detections_dict['detection_scores'])
            for i in range(detections_dict['num_detections']):
                if detections_dict['detection_scores'][i] > 0.02:
                    
                    pred_labels.append(look_up[detections_dict['detection_classes'][i]])
                    n_obj += 1
            cur_pos = 0
            totpred = []
            pred_labelsnew = np.unique(pred_labels)
            
            if type(act) == str:                
                for nm in pred_labelsnew:

                    ac = act.split("'")[1]
                    print(nm, ac)
                    if nm == ac:
                        totpred.append(ac)
                        cur_pos+=1
                accuracy = cur_pos * 100.0
            else:
                for nm in pred_labelsnew:
                    for ac in act:
                        print(nm, ac)
                        ac = ac.split("'")[1]
                        if nm == ac:
                            totpred.append(ac)
                            cur_pos+=1
                accuracy = cur_pos * 100.0 /len(act)
            
            print(accuracy)
            
            writer.writerow({'FileName': name, 'Original_Label': act,
                                            'Predicted_Correct_Label': totpred, 'Accuracy': accuracy})
print('end')

img_0.jpg
headboard stuffed animal
person stuffed animal
printer stuffed animal
stuffed animal stuffed animal
toilet stuffed animal
100.0
img_1.jpg
basket stuffed animal
headboard stuffed animal
paper towel stuffed animal
person stuffed animal
plate stuffed animal
printer stuffed animal
remote control stuffed animal
stuffed animal stuffed animal
toilet stuffed animal
100.0
img_10.jpg
headboard stuffed animal
0.0
img_11.jpg
headboard remote control
person remote control
0.0
img_12.jpg
basket plate
headboard plate
paper towel plate
person plate
plate plate
printer plate
stuffed animal plate
toilet plate
100.0
img_13.jpg
headboard remote control
paper towel remote control
person remote control
printer remote control
stuffed animal remote control
toilet remote control
0.0
img_14.jpg
headboard remote control
0.0
img_15.jpg
basket stuffed animal
dishwasher stuffed animal
headboard stuffed animal
paper towel stuffed animal
person stuffed animal
plate stuffed animal
printer stuffed animal
remo

basket headboard
headboard headboard
paper towel headboard
person headboard
plate headboard
printer headboard
remote control headboard
stuffed animal headboard
100.0
img_70.jpg
basket person
dishwasher person
headboard person
paper towel person
person person
plate person
printer person
remote control person
stuffed animal person
toilet person
100.0
img_71.jpg
basket person
dishwasher person
headboard person
paper towel person
person person
plate person
printer person
remote control person
stuffed animal person
toilet person
100.0
img_72.jpg
basket person
headboard person
paper towel person
person person
plate person
printer person
stuffed animal person
toilet person
100.0
img_73.jpg
basket remote control
dishwasher remote control
headboard remote control
paper towel remote control
person remote control
plate remote control
printer remote control
remote control remote control
stuffed animal remote control
toilet remote control
100.0
img_74.jpg
basket remote control
headboard remote cont

In [30]:
type(act)


str

In [ ]:
print(act)

In [14]:
# to test single image
image_path = "C:/Users/admin-dsvm-cv/Desktop/Computer Vision/TrainData/Data/6.jpg"
detections_dict = my_detector_loaded.score(image_path)

In [40]:
look_up = dict((v,k) for k,v in my_detector.class_map.items())
n_obj = 0
for i in range(detections_dict['num_detections']):
    if detections_dict['detection_scores'][i] > 0.1:
        n_obj += 1
        print("Object {}: label={:11}, score={:.2f}, location=(top: {:.2f}, left: {:.2f}, bottom: {:.2f}, right: {:.2f})".format(
            i, look_up[detections_dict['detection_classes'][i]], 
            detections_dict['detection_scores'][i], 
            detections_dict['detection_boxes'][i][0],
            detections_dict['detection_boxes'][i][1], 
            detections_dict['detection_boxes'][i][2],
            detections_dict['detection_boxes'][i][3]))    
        
print("\nFound {} objects in image {}.".format(n_obj, image_path))           

Object 0: label=wall       , score=0.42, location=(top: 0.00, left: 0.00, bottom: 0.91, right: 0.65)
Object 1: label=wall       , score=0.38, location=(top: 0.00, left: 0.00, bottom: 0.72, right: 0.43)
Object 2: label=wall       , score=0.30, location=(top: 0.48, left: 0.00, bottom: 0.95, right: 1.00)
Object 3: label=wall       , score=0.30, location=(top: 0.00, left: 0.04, bottom: 0.67, right: 1.00)
Object 4: label=wall       , score=0.29, location=(top: 0.54, left: 0.38, bottom: 0.98, right: 1.00)
Object 5: label=wall       , score=0.24, location=(top: 0.02, left: 0.04, bottom: 0.35, right: 0.70)
Object 6: label=wall       , score=0.23, location=(top: 0.21, left: 0.00, bottom: 0.92, right: 0.40)
Object 7: label=wall       , score=0.23, location=(top: 0.40, left: 0.00, bottom: 0.96, right: 0.52)
Object 8: label=wall       , score=0.20, location=(top: 0.01, left: 0.02, bottom: 0.33, right: 0.44)
Object 9: label=wall       , score=0.20, location=(top: 0.06, left: 0.00, bottom: 0.88, rig